<a href="https://colab.research.google.com/github/leonaardoas/Demand-Forecast-/blob/main/Previs%C3%A3o_de_Demanda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn import preprocessing
from sklearn import utils
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error as mae

In [2]:
read = pd.read_csv('/content/drive/MyDrive/Projetos Machine Learning /Previsão de Demanda/Dados/relatorio_01_2020.csv', sep =";", thousands =".", decimal = ",")
read

,Produto,,Código,Localização,Qtde,Qtde faturada,Preço Médio,Valor,Valor faturado,Frete,Desconto,Outras despesas,Total Venda
0,Camisa Manga Curta - Castelo Novo Tamanho:6 a...,,MC-CASTELO6,Estoque,95.0,32.0,45.00,4275.00,1440.00,0.0,0.00,0.0,4275.00
1,Camisa Manga Curta - Castelo Novo Tamanho:10 ...,,MC-CASTELO10,Estoque,89.0,39.0,47.50,4227.50,1852.50,0.0,0.00,0.0,4227.50
2,Camisa Manga Curta - Castelo Novo Tamanho:8 a...,,MC-CASTELO8,Estoque,74.0,27.0,47.50,3515.00,1282.50,0.0,0.00,0.0,3515.00
3,Camisa Manga Curta - Castelo Novo Tamanho:12 ...,,MC-CASTELO12,Estoque,64.0,32.0,47.50,3040.00,1520.00,0.0,4.75,0.0,3035.25
4,Camisa Manga Curta - Castelo Novo Tamanho:14 ...,,MC-CASTELO14,Estoque,51.0,21.0,50.00,2550.00,1050.00,0.0,0.00,0.0,2550.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,Jaqueta Fundamental - Maple Bear Tamanho:16 an...,,JMICF-16,NaN,1.0,0.0,123.80,123.80,0.00,0.0,0.00,0.0,123.80
358,Camisa Pólo - Colégio Bruno Ostmann Tamanho:P,,pol-cbop,Estoque,1.0,0.0,55.00,55.00,0.00,0.0,0.00,0.0,55.00
359,Short Saia Microfibra 100% Poliéster Castelo E...,,ss-castinf4,NaN,1.0,1.0,40.95,40.95,40.95,0.0,0.00,0.0,40.95
360,Camiseta Regata Branca - Colégio Aprovado Tama...,,regb-apv16,Estoque,1.0,1.0,47.00,47.00,47.00,0.0,0.00,0.0,47.00


In [3]:

def carregar_dados(arquivo):
  data = pd.read_csv(f"/content/drive/MyDrive/Projetos Machine Learning /Previsão de Demanda/Dados/{arquivo}", sep=";")
  filename = arquivo.replace(".csv", "")
  filename = filename.split("_")
  month = filename[1]
  year = filename[2]
  data.insert(0, "day", 1) #posição, nome da coluna, elemento a ser inserido
  data.insert(0, "month", month)
  data.insert(0, "year", year)
  data["date"] = pd.to_datetime(data[["year", "month", "day"]])
  data.drop(["day", "month", "year"], axis=1, inplace=True)

  return data

In [4]:
sheets = []

for arquivo in os.listdir("/content/drive/MyDrive/Projetos Machine Learning /Previsão de Demanda/Dados"):
  func = carregar_dados(arquivo)
  sheets.append(func)

dt = pd.concat(sheets, ignore_index=True)
dt


,Produto,,Código,Localização,Qtde,Qtde faturada,Preço Médio,Valor,Valor faturado,Frete,Desconto,Outras despesas,Total Venda,date
0,Camisa Manga Curta - Castelo Novo Tamanho:6 a...,,MC-CASTELO6,Estoque,"95,00","32,00","45,00","4.275,00","1.440,00","0,00","0,00","0,00","4.275,00",2020-01-01
1,Camisa Manga Curta - Castelo Novo Tamanho:10 ...,,MC-CASTELO10,Estoque,"89,00","39,00","47,50","4.227,50","1.852,50","0,00","0,00","0,00","4.227,50",2020-01-01
2,Camisa Manga Curta - Castelo Novo Tamanho:8 a...,,MC-CASTELO8,Estoque,"74,00","27,00","47,50","3.515,00","1.282,50","0,00","0,00","0,00","3.515,00",2020-01-01
3,Camisa Manga Curta - Castelo Novo Tamanho:12 ...,,MC-CASTELO12,Estoque,"64,00","32,00","47,50","3.040,00","1.520,00","0,00","4,75","0,00","3.035,25",2020-01-01
4,Camisa Manga Curta - Castelo Novo Tamanho:14 ...,,MC-CASTELO14,Estoque,"51,00","21,00","50,00","2.550,00","1.050,00","0,00","0,00","0,00","2.550,00",2020-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4152,Camiseta Regata Infantil - CE Carvalho Tamanh...,,regi-car6,Estoque,"2,00","2,00","47,04","94,08","94,08","0,00","0,00","0,00","94,08",2023-04-01
4153,Camiseta Regata Marinho - Colégio Aprovado Tam...,,regm-apv8,Estoque,"1,00","1,00","50,96","50,96","50,96","0,00","0,00","0,00","50,96",2023-04-01
4154,Jaqueta Infantil - Maple Bear Tamanho:4 anos,,JMICI-4,NaN,"2,00","2,00","118,54","237,08","237,08","0,00","0,00","0,00","237,08",2023-04-01
4155,Short Saia - CE Carvalho Tamanho:4 anos,,ssh-car4,NaN,"1,00","1,00","54,88","54,88","54,88","0,00","0,00","0,00","54,88",2023-04-01


In [5]:
totais = dt[dt.Produto == 'Totais']
totais = len(totais)
totais

df = dt[dt.Produto != 'Totais'].reset_index(drop=True)
df
df.info()

null = df.isnull().sum()
null

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4122 entries, 0 to 4121
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Produto          4122 non-null   object        
 1                    4122 non-null   object        
 2   Código           4121 non-null   object        
 3   Localização      2383 non-null   object        
 4   Qtde             4122 non-null   object        
 5   Qtde faturada    4122 non-null   object        
 6   Preço Médio      4122 non-null   object        
 7   Valor            4122 non-null   object        
 8   Valor faturado   4122 non-null   object        
 9   Frete            4122 non-null   object        
 10  Desconto         4122 non-null   object        
 11  Outras despesas  4122 non-null   object        
 12  Total Venda      4122 non-null   object        
 13  date             4122 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(13)
memory 

,0
Produto,0
,0
Código,1
Localização,1739
Qtde,0
Qtde faturada,0
Preço Médio,0
Valor,0
Valor faturado,0
Frete,0


In [6]:
df.drop("Localização", inplace=True, axis=1)
df.drop(df[df["Código"].isnull()].index, inplace=True)
df.reset_index(drop=True, inplace=True)

In [7]:
df

,Produto,,Código,Qtde,Qtde faturada,Preço Médio,Valor,Valor faturado,Frete,Desconto,Outras despesas,Total Venda,date
0,Camisa Manga Curta - Castelo Novo Tamanho:6 a...,,MC-CASTELO6,"95,00","32,00","45,00","4.275,00","1.440,00","0,00","0,00","0,00","4.275,00",2020-01-01
1,Camisa Manga Curta - Castelo Novo Tamanho:10 ...,,MC-CASTELO10,"89,00","39,00","47,50","4.227,50","1.852,50","0,00","0,00","0,00","4.227,50",2020-01-01
2,Camisa Manga Curta - Castelo Novo Tamanho:8 a...,,MC-CASTELO8,"74,00","27,00","47,50","3.515,00","1.282,50","0,00","0,00","0,00","3.515,00",2020-01-01
3,Camisa Manga Curta - Castelo Novo Tamanho:12 ...,,MC-CASTELO12,"64,00","32,00","47,50","3.040,00","1.520,00","0,00","4,75","0,00","3.035,25",2020-01-01
4,Camisa Manga Curta - Castelo Novo Tamanho:14 ...,,MC-CASTELO14,"51,00","21,00","50,00","2.550,00","1.050,00","0,00","0,00","0,00","2.550,00",2020-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4116,Camiseta Regata Amarela - Colégio Ativo Tamanh...,,reg-am-ativo4,"1,00","1,00","47,04","47,04","47,04","0,00","0,00","0,00","47,04",2023-04-01
4117,Camiseta Regata Infantil - CE Carvalho Tamanh...,,regi-car6,"2,00","2,00","47,04","94,08","94,08","0,00","0,00","0,00","94,08",2023-04-01
4118,Camiseta Regata Marinho - Colégio Aprovado Tam...,,regm-apv8,"1,00","1,00","50,96","50,96","50,96","0,00","0,00","0,00","50,96",2023-04-01
4119,Jaqueta Infantil - Maple Bear Tamanho:4 anos,,JMICI-4,"2,00","2,00","118,54","237,08","237,08","0,00","0,00","0,00","237,08",2023-04-01


In [8]:
modelo = df["Produto"].apply(lambda x: x.split("-")[0])
escolaetamanho = df["Produto"].apply(lambda x: x.split("-")[-1])
escolaetamanho

escola = escolaetamanho.apply(lambda x: x.split("Tamanho:")[0])
tamanho = escolaetamanho.apply(lambda x: x.split("Tamanho:")[-1])
tamanho

df.insert(0, "Escola", escola)
df.insert(1, "Modelo", modelo)
df.insert(2, "Tamanho", tamanho)
df.drop("Produto", axis=1, inplace=True)
df


,Escola,Modelo,Tamanho,,Código,Qtde,Qtde faturada,Preço Médio,Valor,Valor faturado,Frete,Desconto,Outras despesas,Total Venda,date
0,Castelo Novo,Camisa Manga Curta,6 anos,,MC-CASTELO6,"95,00","32,00","45,00","4.275,00","1.440,00","0,00","0,00","0,00","4.275,00",2020-01-01
1,Castelo Novo,Camisa Manga Curta,10 anos,,MC-CASTELO10,"89,00","39,00","47,50","4.227,50","1.852,50","0,00","0,00","0,00","4.227,50",2020-01-01
2,Castelo Novo,Camisa Manga Curta,8 anos,,MC-CASTELO8,"74,00","27,00","47,50","3.515,00","1.282,50","0,00","0,00","0,00","3.515,00",2020-01-01
3,Castelo Novo,Camisa Manga Curta,12 anos,,MC-CASTELO12,"64,00","32,00","47,50","3.040,00","1.520,00","0,00","4,75","0,00","3.035,25",2020-01-01
4,Castelo Novo,Camisa Manga Curta,14 anos,,MC-CASTELO14,"51,00","21,00","50,00","2.550,00","1.050,00","0,00","0,00","0,00","2.550,00",2020-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4116,Colégio Ativo,Camiseta Regata Amarela,4 anos,,reg-am-ativo4,"1,00","1,00","47,04","47,04","47,04","0,00","0,00","0,00","47,04",2023-04-01
4117,CE Carvalho,Camiseta Regata Infantil,6 anos,,regi-car6,"2,00","2,00","47,04","94,08","94,08","0,00","0,00","0,00","94,08",2023-04-01
4118,Colégio Aprovado,Camiseta Regata Marinho,8 anos,,regm-apv8,"1,00","1,00","50,96","50,96","50,96","0,00","0,00","0,00","50,96",2023-04-01
4119,Maple Bear,Jaqueta Infantil,4 anos,,JMICI-4,"2,00","2,00","118,54","237,08","237,08","0,00","0,00","0,00","237,08",2023-04-01


In [9]:
escolas = df["Escola"].value_counts()
escolas

,count
Escola,
Maple Bear,1096
Colégio Aprovado,646
Colégio Castelo,435
CE Carvalho,366
Colégio Ativo,278
Colégio Bruno Ostmann,276
Castelo Novo,236
Colégio Aprovado Rio das Ostras,196
Colégio Aprovado Masculina,136


In [10]:
!pip install fuzzywuzzy
from fuzzywuzzy import process

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [11]:
def padronizar_nomes(df, coluna, referencia, limite=80):

    # Cria um dicionário de mapeamento de nomes
    escolapadronizadas = {}

    for nome in df[coluna].unique():
        melhor_correspondencia = process.extractOne(nome, referencia, score_cutoff=limite)
        if melhor_correspondencia:
            melhor_nome, _ = melhor_correspondencia
            escolapadronizadas[nome] = melhor_nome

    # Atualiza a coluna do DataFrame com os nomes padronizados
    df[coluna] = df[coluna].map(escolapadronizadas).fillna(df[coluna])

    return df

# Lista de nomes de referência padronizados
referencia = [
    'Maple Bear',
    'CE Carvalho',
    ' Colégio Castelo',
    'Colégio Aprovado',
    'Colégio Ativo',
    'Colégio Bruno Ostmann',
    'Colégio Radan',
    'Creche Escola Arte de Crescer',
    'Creche Escola Primeiros Passos',
    'Borracha Lapis',
]

# Aplicar a função ao DataFrame
df = padronizar_nomes(df, 'Escola', referencia)

df


,Escola,Modelo,Tamanho,,Código,Qtde,Qtde faturada,Preço Médio,Valor,Valor faturado,Frete,Desconto,Outras despesas,Total Venda,date
0,Castelo Novo,Camisa Manga Curta,6 anos,,MC-CASTELO6,"95,00","32,00","45,00","4.275,00","1.440,00","0,00","0,00","0,00","4.275,00",2020-01-01
1,Castelo Novo,Camisa Manga Curta,10 anos,,MC-CASTELO10,"89,00","39,00","47,50","4.227,50","1.852,50","0,00","0,00","0,00","4.227,50",2020-01-01
2,Castelo Novo,Camisa Manga Curta,8 anos,,MC-CASTELO8,"74,00","27,00","47,50","3.515,00","1.282,50","0,00","0,00","0,00","3.515,00",2020-01-01
3,Castelo Novo,Camisa Manga Curta,12 anos,,MC-CASTELO12,"64,00","32,00","47,50","3.040,00","1.520,00","0,00","4,75","0,00","3.035,25",2020-01-01
4,Castelo Novo,Camisa Manga Curta,14 anos,,MC-CASTELO14,"51,00","21,00","50,00","2.550,00","1.050,00","0,00","0,00","0,00","2.550,00",2020-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4116,Colégio Ativo,Camiseta Regata Amarela,4 anos,,reg-am-ativo4,"1,00","1,00","47,04","47,04","47,04","0,00","0,00","0,00","47,04",2023-04-01
4117,CE Carvalho,Camiseta Regata Infantil,6 anos,,regi-car6,"2,00","2,00","47,04","94,08","94,08","0,00","0,00","0,00","94,08",2023-04-01
4118,Colégio Aprovado,Camiseta Regata Marinho,8 anos,,regm-apv8,"1,00","1,00","50,96","50,96","50,96","0,00","0,00","0,00","50,96",2023-04-01
4119,Maple Bear,Jaqueta Infantil,4 anos,,JMICI-4,"2,00","2,00","118,54","237,08","237,08","0,00","0,00","0,00","237,08",2023-04-01


In [12]:
xx = df["Escola"].value_counts()
xx

,count
Escola,
Maple Bear,1198
Colégio Aprovado,984
Colégio Castelo,466
CE Carvalho,366
Colégio Bruno Ostmann,288
Colégio Ativo,278
Castelo Novo,236
Creche Escola Primeiros Passos,131
Creche Escola Arte de Crescer,85


In [13]:
df['Escola'] = df['Escola'].str.strip().str.lower()
df['Escola'] = df['Escola'].apply(lambda x: "colégio castelo" if x == "castelo novo" else x)
xp = df["Escola"].value_counts()
xp

,count
Escola,
maple bear,1198
colégio aprovado,984
colégio castelo,703
ce carvalho,366
colégio bruno ostmann,288
colégio ativo,278
creche escola primeiros passos,131
creche escola arte de crescer,85
colégio radan,68


In [14]:
borracha = df[
    (df["Escola"] == "Borracha Lapis") |
    (df["Escola"] == "Camisa Manga Curta Fundamental I") |
    (df["Escola"] == "Camisa Manga Curta Fundamental lI") |
    (df["Escola"] == "Modelo Escolar")
]

borracha

# Não foi possível identificar o nome das escolas nem pelo código, portanto optei por remover
remover = ["Borracha Lapis",
           "Camisa Manga Curta Fundamental I",
           "Camisa Manga Curta Fundamental lI",
           "Modelo Escolar"
           ]

df = df.drop(df[df["Escola"].isin(remover)].index)
df.reset_index(drop=True, inplace=True)

df


,Escola,Modelo,Tamanho,,Código,Qtde,Qtde faturada,Preço Médio,Valor,Valor faturado,Frete,Desconto,Outras despesas,Total Venda,date
0,colégio castelo,Camisa Manga Curta,6 anos,,MC-CASTELO6,"95,00","32,00","45,00","4.275,00","1.440,00","0,00","0,00","0,00","4.275,00",2020-01-01
1,colégio castelo,Camisa Manga Curta,10 anos,,MC-CASTELO10,"89,00","39,00","47,50","4.227,50","1.852,50","0,00","0,00","0,00","4.227,50",2020-01-01
2,colégio castelo,Camisa Manga Curta,8 anos,,MC-CASTELO8,"74,00","27,00","47,50","3.515,00","1.282,50","0,00","0,00","0,00","3.515,00",2020-01-01
3,colégio castelo,Camisa Manga Curta,12 anos,,MC-CASTELO12,"64,00","32,00","47,50","3.040,00","1.520,00","0,00","4,75","0,00","3.035,25",2020-01-01
4,colégio castelo,Camisa Manga Curta,14 anos,,MC-CASTELO14,"51,00","21,00","50,00","2.550,00","1.050,00","0,00","0,00","0,00","2.550,00",2020-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4116,colégio ativo,Camiseta Regata Amarela,4 anos,,reg-am-ativo4,"1,00","1,00","47,04","47,04","47,04","0,00","0,00","0,00","47,04",2023-04-01
4117,ce carvalho,Camiseta Regata Infantil,6 anos,,regi-car6,"2,00","2,00","47,04","94,08","94,08","0,00","0,00","0,00","94,08",2023-04-01
4118,colégio aprovado,Camiseta Regata Marinho,8 anos,,regm-apv8,"1,00","1,00","50,96","50,96","50,96","0,00","0,00","0,00","50,96",2023-04-01
4119,maple bear,Jaqueta Infantil,4 anos,,JMICI-4,"2,00","2,00","118,54","237,08","237,08","0,00","0,00","0,00","237,08",2023-04-01


In [15]:
modelosunicos= df["Modelo"].value_counts().sort_values(ascending=False)
modelosunicos[50:]

,count
Modelo,
Camisa Manga Longa Fundamental I,24
Camisa Manga Curta Feminina Fundamental II,20
Jaqueta Fundamental,19
Camiseta Regata Azul,17
Short,17
Camisa Pólo Fundamental,17
Calça Bailarina Fundamental,15
Camiseta Regata Preta,15
Calça Bailarina Infantil,14


In [16]:
df["Modelo"] = df["Modelo"].apply(lambda x: x.strip() and x.lower())
modelosunicos= df["Modelo"].value_counts().sort_values(ascending=False)
modelosunicos[0:50]

,count
Modelo,
camisa manga curta,377
bermuda,303
calça legging,231
camiseta regata,198
camisa manga curta,189
short saia,158
camisa manga curta raglan,134
camisa manga curta infantil,129
camisa manga curta feminino,112


In [17]:
genero = df["Modelo"].apply(lambda x: "M" if "masculin" in x.lower() or "regata" in x.lower() else "F" if "feminin" in x.lower() or "legging" in x.lower() or "vestido" in x.lower() or "bailarina" in x.lower() or "saia" in x.lower() else "U")
qntgenero = genero.value_counts()
qntgenero


df.insert(2, "Genero", genero)
df


,Escola,Modelo,Genero,Tamanho,,Código,Qtde,Qtde faturada,Preço Médio,Valor,Valor faturado,Frete,Desconto,Outras despesas,Total Venda,date
0,colégio castelo,camisa manga curta,U,6 anos,,MC-CASTELO6,"95,00","32,00","45,00","4.275,00","1.440,00","0,00","0,00","0,00","4.275,00",2020-01-01
1,colégio castelo,camisa manga curta,U,10 anos,,MC-CASTELO10,"89,00","39,00","47,50","4.227,50","1.852,50","0,00","0,00","0,00","4.227,50",2020-01-01
2,colégio castelo,camisa manga curta,U,8 anos,,MC-CASTELO8,"74,00","27,00","47,50","3.515,00","1.282,50","0,00","0,00","0,00","3.515,00",2020-01-01
3,colégio castelo,camisa manga curta,U,12 anos,,MC-CASTELO12,"64,00","32,00","47,50","3.040,00","1.520,00","0,00","4,75","0,00","3.035,25",2020-01-01
4,colégio castelo,camisa manga curta,U,14 anos,,MC-CASTELO14,"51,00","21,00","50,00","2.550,00","1.050,00","0,00","0,00","0,00","2.550,00",2020-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4116,colégio ativo,camiseta regata amarela,M,4 anos,,reg-am-ativo4,"1,00","1,00","47,04","47,04","47,04","0,00","0,00","0,00","47,04",2023-04-01
4117,ce carvalho,camiseta regata infantil,M,6 anos,,regi-car6,"2,00","2,00","47,04","94,08","94,08","0,00","0,00","0,00","94,08",2023-04-01
4118,colégio aprovado,camiseta regata marinho,M,8 anos,,regm-apv8,"1,00","1,00","50,96","50,96","50,96","0,00","0,00","0,00","50,96",2023-04-01
4119,maple bear,jaqueta infantil,U,4 anos,,JMICI-4,"2,00","2,00","118,54","237,08","237,08","0,00","0,00","0,00","237,08",2023-04-01


PADRONIZAÇÃO DOS MODELOS DE ROUPAS ATRAVÉS DA LÓGICA FUZZY

In [18]:
def padronizar_modelos(df, coluna, referencia, limite=80):

    # Cria um dicionário de mapeamento de modelos
    modelospadronizados = {}

    for nome in df[coluna].unique():
        melhor_correspondencia = process.extractOne(nome, referencia, score_cutoff=limite)
        if melhor_correspondencia:
            melhor_nome, _ = melhor_correspondencia
            modelospadronizados[nome] = melhor_nome

    # Atualiza a coluna do DataFrame com os modelos padronizados
    df[coluna] = df[coluna].map(modelospadronizados).fillna(df[coluna])

    return df

# Lista de modelos de referência padronizados
referencia = [
    'Maple Bear',
    'CE Carvalho',
    ' Colégio Castelo',
    'Colégio Aprovado',
    'Colégio Ativo',
    'Colégio Bruno Ostmann',
    'Colégio Radan',
    'Creche Escola Arte de Crescer',
    'Creche Escola Primeiros Passos',
    'Borracha Lapis',
]

# Aplicar a função ao DataFrame
df = padronizar_modelos(df, 'Modelo', referencia)

df

,Escola,Modelo,Genero,Tamanho,,Código,Qtde,Qtde faturada,Preço Médio,Valor,Valor faturado,Frete,Desconto,Outras despesas,Total Venda,date
0,Colégio Castelo,camisa manga curta,U,6 anos,,MC-CASTELO6,"95,00","32,00","45,00","4.275,00","1.440,00","0,00","0,00","0,00","4.275,00",2020-01-01
1,Colégio Castelo,camisa manga curta,U,10 anos,,MC-CASTELO10,"89,00","39,00","47,50","4.227,50","1.852,50","0,00","0,00","0,00","4.227,50",2020-01-01
2,Colégio Castelo,camisa manga curta,U,8 anos,,MC-CASTELO8,"74,00","27,00","47,50","3.515,00","1.282,50","0,00","0,00","0,00","3.515,00",2020-01-01
3,Colégio Castelo,camisa manga curta,U,12 anos,,MC-CASTELO12,"64,00","32,00","47,50","3.040,00","1.520,00","0,00","4,75","0,00","3.035,25",2020-01-01
4,Colégio Castelo,camisa manga curta,U,14 anos,,MC-CASTELO14,"51,00","21,00","50,00","2.550,00","1.050,00","0,00","0,00","0,00","2.550,00",2020-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4116,Colégio Ativo,camiseta regata amarela,M,4 anos,,reg-am-ativo4,"1,00","1,00","47,04","47,04","47,04","0,00","0,00","0,00","47,04",2023-04-01
4117,CE Carvalho,camiseta regata infantil,M,6 anos,,regi-car6,"2,00","2,00","47,04","94,08","94,08","0,00","0,00","0,00","94,08",2023-04-01
4118,Colégio Aprovado,camiseta regata marinho,M,8 anos,,regm-apv8,"1,00","1,00","50,96","50,96","50,96","0,00","0,00","0,00","50,96",2023-04-01
4119,Maple Bear,jaqueta infantil,U,4 anos,,JMICI-4,"2,00","2,00","118,54","237,08","237,08","0,00","0,00","0,00","237,08",2023-04-01
